# CPOL radar

CPOL is a dual-polarisation C-band radar that have been producing data since 1998. It has been referenced by a 100+ publications and it is actively used by a large community in Australia and around the world. CPOL is unique, as it is the only dual-polarisation radar in the Tropics. It has been used by NASA to calibrate the satellite TRMM.

![cpol](img/cpol.jpg)

The aim of this notebook is to get a hand on CPOL data and discover what's inside and how to represent of manipulate the data.

In [ ]:
%matplotlib inline
import os
import datetime
import warnings

import pyart
import requests
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.colors import LogNorm

In [ ]:
warnings.simplefilter("ignore")
sns.set_style("whitegrid")

In [ ]:
def download_cpol_data(date: datetime.datetime, ppi: bool=True) -> str:
    """
    Download CPOL data for given date. 
    
    Parameters:
    ===========
    date: str or datetime or pd.Timestamp
        Date time for which we want the CPOL data
    ppi: bool
        True for downloading the PPIs and False for downloading the gridded data.
    """
    date = pd.Timestamp(date)
    if date.minute % 10 != 0:
        raise ValueError("CPOL data is every 10 minutes.")
    year = date.year
    datestr = date.strftime("%Y%m%d")
    datetimestr = date.strftime("%Y%m%d.%H%M")
    if ppi:
        url = f"http://dapds00.nci.org.au/thredds/fileServer/hj10/cpol/cpol_level_1b/v2020/ppi/{year}/{datestr}/twp10cpolppi.b1.{datetimestr}00.nc"
    else:
        url = f"http://dapds00.nci.org.au/thredds/fileServer/hj10/cpol/cpol_level_1b/v2020/gridded/grid_150km_2500m/{year}/{datestr}/twp10cpolppi.b1.{datetimestr}00.nc"
    
    fname = os.path.basename(url)
    try:
        os.mkdir("dwl")
    except FileExistsError:
        pass
    outfilename = os.path.join("dwl", fname)
    if os.path.isfile(outfilename):
        print("Output file already exists, doing nothing")
        return outfilename
    
    r = requests.get(url)
    try:
        r.raise_for_status()
    except Exception:
        print("No file found for this date. CPOL ran from 1998-12-6 to 2017-5-2, wet season only. Try another date.")
        return None
    
    with open(outfilename, "wb") as fid:
        fid.write(r.content)

    return outfilename

In [ ]:
date = datetime.datetime(2014, 2, 18, 20, 0)

In [ ]:
filename = download_cpol_data(date)

Let's open file with Py-ART.

Py-ART is a Python library that helps with handeling radar data. Radars have been running for a long time, and each manufacturer, and each era have their different file formats. Jungleling between file format is difficult, and Py-ART helps do that, it reads and convert everything to the CF/Radial convention.

Py-ART also provide a set of correction, retrievals and utilities. It's not just about the input/output of data!

In [ ]:
radar = pyart.io.read(filename)

Let's look at the documentation. What is the variable defined as "radar"?

In [ ]:
radar?

We see that the object "radar" contains a lot of attributes. The first 3 ones are:

time : dict

    Time at the center of each ray.
    
range : dict

    Range to the center of each gate (bin).
    
fields : dict of dicts

    Moment fields.

In [ ]:
# Let's see what fields are present in the radar:
print("\n".join(radar.fields.keys()))

In [ ]:
gr = pyart.graph.RadarDisplay(radar)

In [ ]:
gr.plot_ppi("corrected_reflectivity", vmin=-15, vmax=65)
plt.show()

# A bit of theory

The image above is a PPI. A plan position indicator (PPI) is a type of radar display that represents the radar antenna in the center of the display, with the distance from it and height above ground drawn as concentric circles. As the radar antenna rotates, a radial trace on the PPI sweeps in unison with it about the center point. It is the most common type of radar display.

The radar antenna sends pulses while rotating 360 degrees around the radar site at a fixed elevation angle. It can then change angle or repeat at the same angle according to the need. Return echoes from targets are received by the antenna and processed by the receiver and the most direct display of those data is the PPI. From the figure above, can you tell what was elevation angle of the PPI?

By doing measurements at different elevation angles, we obtain a 3D volumetric scan of the atmosphere.

In [ ]:
elevations = np.round([radar.elevation['data'][i].mean() for i in radar.iter_slice()], 1)

In [ ]:
elevations

In [ ]:
# Let's see how the radar perform the volumetric scan of the atmosphere

r = np.linspace(0, 150)  # radar range here goes to 150 km
earth_radius = 6371  # km, we want to correct for the earth sphericity, cause it's not flat.
beamwidth = 1 # Radar beamwidth in degree

In [ ]:
fig = plt.figure(figsize=(12, 7))
for e in elevations:
    y1 =  r * np.tan(np.pi * (e - beamwidth / 2) / 180) + np.sqrt(r ** 2 + earth_radius ** 2) - earth_radius
    y2 =  r * np.tan(np.pi * (e + beamwidth / 2) / 180) + np.sqrt(r ** 2 + earth_radius ** 2) - earth_radius
    y =   r * np.tan(np.pi * (e) / 180) + np.sqrt(r ** 2 + earth_radius ** 2) - earth_radius
    plt.fill_between(r, y1, y2, alpha=0.5, edgecolor="k")
    plt.plot(r, y, label=f"{e:0.3}$^\circ$")
    
plt.legend()
plt.yticks(np.arange(0, 20.1, 1))
plt.ylim(0, 20)
plt.xlabel("Range (km)")
plt.ylabel("Altitude Above Ground Level (km)")
plt.title("Radar scan - elevation")
fig.tight_layout()

plt.show()

Now let's plot all the radar sweeps, i.e. all the scans at a fixed elevation angle

In [ ]:
radar.nsweeps  # That's the total number of elevation 

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(15, 20))
ax = ax.ravel()

for i in range(radar.nsweeps):
    gr.plot_ppi("corrected_reflectivity", sweep=i, vmin=-15, vmax=65, ax=ax[i])
    gr.plot_range_rings([50, 100, 150], ax=ax[i], lw=1)
    ax[i].set_aspect(1)
    
fig.tight_layout()
plt.show()

Another interesting way of representing the data is to do a vertical cross-section along a specified azimuth:

In [ ]:
xsect = pyart.util.cross_section_ppi(radar, [45, 90])
display = pyart.graph.RadarDisplay(xsect)

fig, ax = plt.subplots(2, 1, figsize=(12, 7))
ax = ax.ravel()

display.plot('corrected_reflectivity', 0, vmin=-15, vmax=65, ax=ax[0])
display.plot('corrected_reflectivity', 1, vmin=-15, vmax=65, ax=ax[1])

for a in ax:
    a.set_ylim(0, 16)

fig.tight_layout()
plt.show()

But wait! There's an interesting structure on the PPI North-West. How can I do a cross section of it? What's the azimuth?

In [ ]:
azimuths = np.arange(0, 360, 20)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
gr.plot_ppi("corrected_reflectivity", sweep=0, ax=ax, vmin=-15, vmax=65)
gr.plot_range_rings([50, 100, 150], ax=ax, lw=1)

for a in azimuths:
    theta = np.deg2rad((450 - a) % 360)
    ax.plot([0, 150 * np.cos(theta)], [0, 150 * np.sin(theta)], label=f"{a}$^\circ$")
    
plt.legend()
ax.set_aspect(1)
fig.tight_layout()
plt.show()

300 degree azimuth!

In [ ]:
xsect = pyart.util.cross_section_ppi(radar, [100, 300])
display = pyart.graph.RadarDisplay(xsect)

fig, ax = plt.subplots(2, 1, figsize=(12, 7))
ax = ax.ravel()

display.plot('corrected_reflectivity', 0, vmin=-15, vmax=65, ax=ax[0])
display.plot('corrected_reflectivity', 1, vmin=-15, vmax=65, ax=ax[1])

for a in ax:
    a.set_ylim(0, 16)

fig.tight_layout()
plt.show()

Let's look at the difference between the corrected and uncorrected reflectivity. Can you spot the differences?

In [ ]:
xsect = pyart.util.cross_section_ppi(radar, [300])
display = pyart.graph.RadarDisplay(xsect)

fig, ax = plt.subplots(2, 1, figsize=(12, 7))
ax = ax.ravel()

display.plot('corrected_reflectivity', 0, vmin=-15, vmax=65, ax=ax[0], cmap="pyart_NWSRef")
display.plot('total_power', 0, vmin=-15, vmax=65, ax=ax[1], cmap="pyart_NWSRef")

for a in ax:
    a.set_ylim(0, 16)

fig.tight_layout()
plt.show()

In [ ]:
# Just creating a custom colormap, you can disregard this cell and go to the next
def _adjust_csu_scheme_colorbar_for_pyart(cb):
    """
    Generate colorbar for the hydrometeor classification.
    """
    cb.set_ticks(np.arange(0.55, 10, 0.9))
    cb.ax.set_yticklabels(
        [
            "None",
            "Drizzle",
            "Rain",
            "Ice Crystals",
            "Aggregates",
            "Wet/Melting Snow",
            "Vertically Aligned Ice",
            "Low-Density Graupel",
            "High-Density Graupel",
            "Hail",
            "Big Drops",
        ]
    )
    cb.ax.set_ylabel("")
    cb.ax.tick_params(length=0)
    return cb

# echo classification
# create colormap
hca_colors = [
    "White",
    "LightBlue",
    "SteelBlue",
    "MediumBlue",
    "Plum",
    "MediumPurple",
    "m",
    "Green",
    "YellowGreen",
    "Gold",
    "Red",
]
hca_cmap = colors.ListedColormap(hca_colors)

In [ ]:
gr = pyart.graph.RadarDisplay(radar)
radar.fields['velocity']['standard_name'] = 'Velocity'
radar.fields['corrected_velocity']['standard_name'] = 'Corrected_Velocity'

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(16, 12), sharex=True, sharey=True, constrained_layout=True)
ax = ax.flatten()

gr.plot_ppi("corrected_reflectivity", ax=ax[0], cmap="pyart_NWSRef")
gr.plot_ppi("radar_estimated_rain_rate", ax=ax[1], norm=LogNorm(1e-2, 1e2))
gr.plot_ppi("radar_echo_classification", ax=ax[2], cmap=hca_cmap, vmin=0, vmax=10)
gr.cbs[2] = _adjust_csu_scheme_colorbar_for_pyart(gr.cbs[2])
gr.plot_ppi("corrected_differential_reflectivity", ax=ax[3])
gr.plot_ppi("corrected_differential_phase", ax=ax[4], vmin=-180, vmax=180, cmap="pyart_Wild25",)
gr.plot_ppi("corrected_specific_differential_phase", ax=ax[5], vmin=-2, vmax=5, cmap="pyart_Theodore16",)
gr.plot_ppi("velocity", ax=ax[6], cmap="pyart_BuDRd18", vmin=-30, vmax=30)
gr.plot_ppi("corrected_velocity", ax=ax[7], cmap="pyart_BuDRd18", vmin=-30, vmax=30,)
gr.plot_ppi("cross_correlation_ratio", ax=ax[8], vmin=0.5, vmax=1.05)

for a in ax:
    gr.plot_range_rings([50, 100, 150], ax=a, lw=1, col="#CDCDCD")
    a.set_aspect(1)
    a.set_xlim(-150, 150)
    a.set_ylim(-150, 150)
    
plt.show()

In [ ]:
xsect = pyart.util.cross_section_ppi(radar, [100, 300])
display = pyart.graph.RadarDisplay(xsect)

fig, ax = plt.subplots(3, 1, figsize=(12, 11))
ax = ax.ravel()

display.plot('corrected_reflectivity', 1, vmin=-15, vmax=65, cmap="pyart_NWSRef", ax=ax[0])
display.plot('corrected_specific_differential_phase', 1, vmin=-2, vmax=5, cmap="pyart_Theodore16", ax=ax[1])
display.plot('radar_echo_classification', 1, cmap=hca_cmap, vmin=0, vmax=10, ax=ax[2])
display.cbs[2] = _adjust_csu_scheme_colorbar_for_pyart(display.cbs[2])

for a in ax:
    a.set_ylim(0, 16)
    
fig.tight_layout()
plt.show()